In [19]:
import xml.etree.ElementTree as ET
from tilespec import TileSpec,Transform,AffineModel
from renderapi import Render
import os
import json
import numpy as np

In [20]:
inputfile='/nas/data/M247514_Rorb_1/scripts/test/out_edit2.xml'
prealignedOwner = 'Sharmishtaas'
prealignedProject = 'M247514_Rorb_1'
prealignedStack = 'REGISTERED_FLATFIELD_FIX_MBP'

postalignedOwner = prealignedOwner
postalignedProject = prealignedProject
postalignedStack = 'ALIGNED_MBP'

inputOwner = prealignedOwner
inputProject = prealignedProject
inputPrefix = 'REGISTERED_FLATFIELD_FIX_'

outputOwner = prealignedOwner
outputProject = prealignedProject
outputPrefix = 'ALIGNED2_'

outputDir = '/nas/data/M247514_Rorb_1/processed/aligned_tilespecs'
host = 'ibs-forrestc-ux1.corp.alleninstitute.org'
port = 8081
render = Render(host,port,inputOwner,inputProject)

In [21]:
inputStacks = [st for st in render.get_stacks_by_owner_project(inputOwner,inputProject) if st.startswith(inputPrefix)]

http://ibs-forrestc-ux1.corp.alleninstitute.org:8081/render-ws/v1/owner/Sharmishtaas/stacks/


In [24]:
def define_aligned_tilespecs(prealignedStack,postalignedStack,inputStack):
    zvalues = render.get_z_values_for_stack(inputStack)
    #zvalues = [22.0]
    finaltilespecs = []
    for z in zvalues:
        prealignedTS = render.get_tile_specs_from_z(prealignedStack,z,owner=prealignedOwner)[7]
        postalignedTS = render.get_tile_specs_from_z(postalignedStack,z,owner=postalignedOwner)
        postalignedTS =[ts for ts in postalignedTS if ts.tileId==prealignedTS.tileId][0]
        tform_W_to_R = prealignedTS.tforms

        tform_R_to_W = list(tform_W_to_R)
        tform_R_to_W.reverse()
        tform_R_to_W = [tf.invert() for tf in tform_R_to_W]

        tform_W_to_A = postalignedTS.tforms

        tform_R_to_A = tform_R_to_W + tform_W_to_A
        #print tform_R_to_A

        inputTS = render.get_tile_specs_from_z(inputStack,z,owner=inputOwner)
        for ts in inputTS:
            ts.tforms+=tform_R_to_A
            finaltilespecs.append(ts)
    return finaltilespecs

In [25]:
for inputStack in inputStacks:
    inputchannel = inputStack.replace(inputPrefix,'')
    outputStack = outputPrefix + inputchannel
    print inputStack,outputStack
    
    finaltilespecs = define_aligned_tilespecs(prealignedStack,postalignedStack,inputStack)
    json_tilespecs=[ts.to_dict() for ts in finaltilespecs]
    if not os.path.isdir(outputDir):
        os.makedirs(outputDir)
    outjson = os.path.join(outputDir,'%s_%s_%s.json'%(inputOwner,inputProject,outputStack))
    json.dump(json_tilespecs,open(outjson,'w'),indent=4)
    print outputStack,outjson
    render.delete_stack(outputStack,owner=outputOwner,project=outputProject)
    render.create_stack(outputStack,owner=outputOwner,project=outputProject)
    
    render.import_jsonfiles(outputStack,[outjson],owner=outputOwner,project=outputProject)

REGISTERED_FLATFIELD_FIX_MBP ALIGNED2_MBP
http://ibs-forrestc-ux1.corp.alleninstitute.org:8081/render-ws/v1/owner/Sharmishtaas/project/M247514_Rorb_1/stack/REGISTERED_FLATFIELD_FIX_MBP/zValues/
ALIGNED2_MBP /nas/data/M247514_Rorb_1/processed/aligned_tilespecs/Sharmishtaas_M247514_Rorb_1_ALIGNED2_MBP.json

http://ibs-forrestc-ux1.corp.alleninstitute.org:8081/render-ws/v1/owner/Sharmishtaas/project/M247514_Rorb_1/stack/ALIGNED2_MBP/state/LOADING
http://ibs-forrestc-ux1.corp.alleninstitute.org:8081/render-ws/v1/owner/Sharmishtaas/project/M247514_Rorb_1/stack/ALIGNED2_MBP/state/COMPLETE
REGISTERED_FLATFIELD_FIX_DAPI_3 ALIGNED2_DAPI_3
http://ibs-forrestc-ux1.corp.alleninstitute.org:8081/render-ws/v1/owner/Sharmishtaas/project/M247514_Rorb_1/stack/REGISTERED_FLATFIELD_FIX_DAPI_3/zValues/
ALIGNED2_DAPI_3 /nas/data/M247514_Rorb_1/processed/aligned_tilespecs/Sharmishtaas_M247514_Rorb_1_ALIGNED2_DAPI_3.json

http://ibs-forrestc-ux1.corp.alleninstitute.org:8081/render-ws/v1/owner/Sharmishtaas/pro